# 9. 프로젝트 - Movielens 영화 추천 실습

## Movielens 데이터셋을 활용한 영화 추천시스템
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. `MovieLens 1M Dataset` 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

#### 실습 목표
- 추천시스템의 개념과 목적을 이해한다.
- Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
- 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
- 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
- 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
- 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.

#### 목차
1) 데이터 준비와 전처리  
2) 분석해 봅시다.  
3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.  
4) CSR matrix를 직접 만들어 봅시다.  
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.  
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

### 추천시스템이란?
- **협업 필터링(Collaborative Filtering) 방식**  
기존 사용자 행동 정보를 분석하여 해당 사용자와 비슷한 성향의 사용자들이 기존에 좋아했던 항목을 추천하는 기술 (아이템과 사용자 간의 행동 또는 관계에만 주목)
- **콘텐츠 기반 필터링(Contents-based Filtering) 방식**  
항목 자체를 분석하여 항목 간의 유사성을 파악하여 추천을 구현 (아이템 자체의 속성에만 주목)

**협업 필터링을 바로 사용할 수 없게 만드는 세 가지 제약조건**  
1. 콜드 스타트(Cold Start) : 기존의 자료를 기반으로 추천을 제공하므로 기존에 없던 새로운 항목이 추가되는 경우 추천이 곤란해진다
2. 계산량이 비교적 많은 알고리즘이므로 사용자 수가 많은 경우 효율적으로 추천할 수 없다
3. 롱테일(Long tail) : 사용자들의 관심이 적은 다수의 항목은 추천을 위한 충분한 정보를 제공하지 못하는 경우가 많다

## 1) 데이터 준비와 전처리

### 데이터 준비

Movielens 데이터는 `rating.dat` 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 정리되어 있다.

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings = pd.merge(ratings, movies)
ratings = ratings[["user_id", "movie_id", "title", "count"]]
ratings.sort_values("user_id")

,user_id,movie_id,title,count
0,1,1193,One Flew Over the Cuckoo's Nest (1975),5
31113,1,2294,Antz (1998),4
31674,1,3186,"Girl, Interrupted (1999)",4
32044,1,1566,Hercules (1997),4
32415,1,588,Aladdin (1992),4
...,...,...,...,...
657728,6040,334,Vanya on 42nd Street (1994),4
393446,6040,1294,M*A*S*H (1970),4
253075,6040,994,Big Night (1996),3
127665,6040,2396,Shakespeare in Love (1998),3


## 2) 분석해 봅시다.

In [7]:
# 영화 수
print('ratings에 있는 유니크한 영화 개수 :', ratings['movie_id'].nunique())
# 유저 수
print('ratings에 있는 유니크한 사용자 수 :', ratings['user_id'].nunique())

ratings에 있는 유니크한 영화 개수 : 3628
ratings에 있는 유니크한 사용자 수 : 6039


In [8]:
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

#### 검색을 쉽게하기 위해 title 문자열 소문자로 바꿔준다.

In [9]:
ratings['title'] = ratings['title'].str.lower()
ratings.head(10)

,user_id,movie_id,title,count
0,1,1193,one flew over the cuckoo's nest (1975),5
1,2,1193,one flew over the cuckoo's nest (1975),5
2,12,1193,one flew over the cuckoo's nest (1975),4
3,15,1193,one flew over the cuckoo's nest (1975),4
4,17,1193,one flew over the cuckoo's nest (1975),5
5,18,1193,one flew over the cuckoo's nest (1975),4
6,19,1193,one flew over the cuckoo's nest (1975),5
7,24,1193,one flew over the cuckoo's nest (1975),5
8,28,1193,one flew over the cuckoo's nest (1975),3
9,33,1193,one flew over the cuckoo's nest (1975),5


In [10]:
ratings

,user_id,movie_id,title,count
0,1,1193,one flew over the cuckoo's nest (1975),5
1,2,1193,one flew over the cuckoo's nest (1975),5
2,12,1193,one flew over the cuckoo's nest (1975),4
3,15,1193,one flew over the cuckoo's nest (1975),4
4,17,1193,one flew over the cuckoo's nest (1975),5
...,...,...,...,...
836473,5851,3607,one little indian (1973),5
836474,5854,3026,slaughterhouse (1987),4
836475,5854,690,"promise, the (versprechen, das) (1994)",3
836476,5938,2909,"five wives, three secretaries and me (1998)",4


In [12]:
# 고유한 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
# print(user_to_idx['wooil'])
print(movie_to_idx['matrix, the (1999)'])
print(movie_to_idx['terminator 2: judgment day (1991)'])
print(movie_to_idx['shining, the (1980)'])
print(movie_to_idx['clockwork orange, a (1971)'])
print(movie_to_idx['400 blows, the (les quatre cents coups) (1959)'])

124
92
993
904
784


In [15]:
my_favorite_title = ['matrix, the (1999)' , 'terminator 2: judgment day (1991) ' ,'shining, the (1980)' ,'clockwork orange, a (1971)' ,'400 blows, the (les quatre cents coups) (1959)']
my_favorite_id = [124 , 92, 993, 904, 784]

my_movielist = pd.DataFrame({'user_id': ['6039']*5, 'title': my_favorite_title, 'movie_id': my_favorite_id, 'count':[5]*5})
my_movielist

,user_id,title,movie_id,count
0,6039,"matrix, the (1999)",124,5
1,6039,terminator 2: judgment day (1991),92,5
2,6039,"shining, the (1980)",993,5
3,6039,"clockwork orange, a (1971)",904,5
4,6039,"400 blows, the (les quatre cents coups) (1959)",784,5


In [16]:
if not ratings.isin({'user_id':['6039']})['user_id'].any():
    ratings = ratings.append(my_movielist, ignore_index=True)

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,title,count
836473,5851,3607,one little indian (1973),5
836474,5854,3026,slaughterhouse (1987),4
836475,5854,690,"promise, the (versprechen, das) (1994)",3
836476,5938,2909,"five wives, three secretaries and me (1998)",4
836477,5948,1360,identification of a woman (identificazione di ...,5
836478,6039,124,"matrix, the (1999)",5
836479,6039,92,terminator 2: judgment day (1991),5
836480,6039,993,"shining, the (1980)",5
836481,6039,904,"clockwork orange, a (1971)",5
836482,6039,784,"400 blows, the (les quatre cents coups) (1959)",5


In [20]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [21]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna() # .dropna() : 결측치(누락데이터) 제거
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 주기
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 주기
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie id column indexing Fail!!')

ratings


user_id column indexing OK!!
movie id column indexing OK!!


,user_id,movie_id,title,count
0,0,0,one flew over the cuckoo's nest (1975),5
1,1,0,one flew over the cuckoo's nest (1975),5
2,2,0,one flew over the cuckoo's nest (1975),4
3,3,0,one flew over the cuckoo's nest (1975),4
4,4,0,one flew over the cuckoo's nest (1975),5
...,...,...,...,...
836478,6039,2901,"matrix, the (1999)",5
836479,6039,2656,terminator 2: judgment day (1991),5
836480,6039,2061,"shining, the (1980)",5
836481,6039,569,"clockwork orange, a (1971)",5


In [22]:
ratings.head(30)

,user_id,movie_id,title,count
0,0,0,one flew over the cuckoo's nest (1975),5
1,1,0,one flew over the cuckoo's nest (1975),5
2,2,0,one flew over the cuckoo's nest (1975),4
3,3,0,one flew over the cuckoo's nest (1975),4
4,4,0,one flew over the cuckoo's nest (1975),5
5,5,0,one flew over the cuckoo's nest (1975),4
6,6,0,one flew over the cuckoo's nest (1975),5
7,7,0,one flew over the cuckoo's nest (1975),5
8,8,0,one flew over the cuckoo's nest (1975),3
9,9,0,one flew over the cuckoo's nest (1975),5


## 4) CSR matrix를 직접 만들어 봅시다.

In [23]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id'])), shape=(num_user, num_movie))
csr_data


<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [25]:

from implicit.als import AlternatingLeastSquares 
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [28]:
als_model = AlternatingLeastSquares(factors=3000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [29]:
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [46]:
# wooil, matrix = user_to_idx[6039], movie_to_idx['clockwork orange, a (1971)']
wooil_vector, matrix_vector = als_model.user_factors[6039], als_model.item_factors[124]

In [47]:
wooil_vector

array([-0.28510338,  0.02922645, -0.21891011, ...,  0.06495595,
       -0.18433803,  0.3407063 ], dtype=float32)

In [48]:
matrix_vector

array([0.00741206, 0.00663579, 0.00445783, ..., 0.01332289, 0.00640115,
       0.01749524], dtype=float32)

In [49]:
np.dot(wooil_vector, matrix_vector)

-0.0023709978

||  <center>**평가 문항**</center> |  <center>**상세 기준**</center> |
|:--------|:--------|:--------|
|**1**|CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.|
|**2**|MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.|
|**3**|비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.|

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [65]:
def id_to_title(id):
    return movies['title'][movies['movie_id']==id].values[0]

In [66]:
favorite_movie = 124
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(2901, 0.99999994),
 (3573, 0.8027864),
 (3578, 0.7983731),
 (3583, 0.796636),
 (3597, 0.7930495),
 (3598, 0.792957),
 (3448, 0.79079014),
 (3373, 0.7902386),
 (3250, 0.78878605),
 (3579, 0.78838694),
 (3580, 0.78789157),
 (3533, 0.7876267),
 (3563, 0.7870171),
 (3525, 0.78625613),
 (3552, 0.78600115)]

In [72]:
[id_to_title(i[0]) for i in similar_movie]

['Phantasm (1979)',
 'Carnosaur 2 (1995)',
 'Gladiator (2000)',
 'Black Tights (Les Collants Noirs) (1960)',
 'Whipped (2000)',
 'Hamlet (2000)',
 'Good Morning, Vietnam (1987)',
 'Buck and the Preacher (1972)',
 'Alive (1993)',
 'I Dreamed of Africa (2000)',
 'Up at the Villa (2000)',
 "Actor's Revenge, An (Yukinojo Henge) (1963)",
 'Crow: Salvation, The (2000)',
 'Bachelor Party (1984)',
 'Caddyshack (1980)']

In [61]:
def get_similar_movie(movie_name: str):
    movie_name = movie_name
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [id_to_title(i[0]) for i in similar_movie]
    return similar_movie

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [73]:
user = user_to_idx[6039]

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2889, 0.061023943),
 (3120, 0.04686772),
 (1455, 0.044885553),
 (3384, 0.044112217),
 (3163, 0.043517146),
 (658, 0.042995892),
 (495, 0.040670626),
 (2970, 0.040381417),
 (1951, 0.0394252),
 (3396, 0.03889598),
 (2040, 0.038895298),
 (2755, 0.038418274),
 (3481, 0.037549306),
 (1398, 0.037543938),
 (3389, 0.037118264),
 (3470, 0.035241086),
 (2032, 0.03443508),
 (3041, 0.03425416),
 (3326, 0.033923164),
 (1342, 0.03358918)]

In [74]:
[id_to_title(i[0]) for i in movie_recommended]

['Mystery, Alaska (1999)',
 'Distinguished Gentleman, The (1992)',
 'Hotel de Love (1996)',
 'Taking of Pelham One Two Three, The (1974)',
 'Topsy-Turvy (1999)',
 "Billy's Holiday (1995)",
 'In the Realm of the Senses (Ai no corrida) (1976)',
 'Fitzcarraldo (1982)',
 'Oliver! (1968)',
 'Muppet Movie, The (1979)',
 'Computer Wore Tennis Shoes, The (1970)',
 'Light of Day (1987)',
 'High Fidelity (2000)',
 'In Love and War (1996)',
 "Let's Get Harry (1986)",
 'Dersu Uzala (1974)',
 'Barefoot Executive, The (1971)',
 'Meatballs Part II (1984)',
 'What Planet Are You From? (2000)',
 'Candyman (1992)']

## 9) 추천시스템에 기여한 정도 확인

In [75]:
Matrix = movie_to_idx[124]
# 이 explain method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됨)를 반환
explain = als_model.explain(user, csr_data, itemid=Matrix)

In [77]:
explain
[(id_to_title(i[0]), i[1]) for i in explain[1]]

[('Vampires (1998)', 0.019935724973712845),
 ('Last of the Mohicans, The (1992)', 0.009199923774342363),
 ('American President, The (1995)', 0.006322443728450511),
 ('George of the Jungle (1997)', 0.00625513129016633),
 ('Jungle Book, The (1994)', 0.004759204537540321),
 ('Being There (1979)', 0.003944598852670939),
 ('Century (1993)', 0.003762114737549159),
 ('Shine (1996)', 0.0033378380337845096),
 ('Little Big League (1994)', 0.003091450745184624),
 ("It's a Wonderful Life (1946)", 0.0028359694452717863)]

## 회고

추천시스템에 대해 공부해보고, 실제 내가 선호하는 영화나 음악을 추천해서 보여주는 과정이 흥미로웠는데, 나의 선호 영화 목록 리스트를 추가하는 부분에서 인덱싱하는 과정에서 데이터가 자꾸 망가지는 것 같아 그 부분을 해결해보려다가 시간을 너무 많이 잡아먹기도 했고, 힘을 다 소진해버려서 노드를 완성하는 데 있어서 만족스럽지 못한 부분이 많았던 것 같아 아쉬웠다. 시간이 좀더 있을 때, 인덱싱을 제대로 잘 해서 재미있는 영화를 많이 추천받아 볼 수 있으면 좋을 것 같다.